# Median Metrics Tables

F1/FX

This notebook was written by Logan Qualls. Data for this work is sourced from the National Center for Atmospheric Research's Catchment Attributes and Meterology for Large-Sample Studies (CAMELS) dataset, and this notebook is designed to work specifically with Frederik Kratzert's NeuralHydrology (NH; https://github.com/neuralhydrology/neuralhydrology) and Grey Nearing's SACSMA-SNOW17 (SAC-SMA; https://github.com/Upstream-Tech/SACSMA-SNOW17). NH provides a flexible framework with a variety of tools specifically designed for straightforward application of Long Short-Term Memory networks to hydrological modeling. The SACSMA-SNOW17 model provides a Python interface for the SAC-SMA model.

This notebook compiles all the median metrics for all avaliable models into several tables by type (extreme or random) and ultimately combines them for export.

### Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import pickle as pkl
from pathlib import Path
import matplotlib.pyplot as plt

Here I remove the display limit for the rows and columns for Pandas DataFrames.

In [2]:
#View all columns
pd.set_option('display.max_columns', None)
#View all rows
pd.set_option('display.max_rows', None)

### Define Parameters

The only parameters necessary for this notebook are the desired climate index and the path to the notebook_env_saves folder, if that folder has been moved from the original folder structure.



##### Notebook Configurations

In [3]:
#########################################################################################

#Define index we want to create tables and histograms for (currently only supports one)
index = 'aridity'

#########################################################################################

##### Paths

In [4]:
#########################################################################################

#Path to working directory (current directory)
working_dir = Path(os.getcwd())

#Path to notebook_env_saves directory (.../notebook_env_saves)
env_saves_dir = working_dir / 'notebook_env_saves'

#########################################################################################

### Load Source Data

First we load in our source data. This cell automatically retrieves a list of files starting with 'basin', indicating that the file contains per-basin metrics (as opposed to overall, as for the case in the files starting with 'cdfs') for a model. Assuming you have ensembled and analyzed a model prior to running this notebook, it will be included in the resulting tables.

Here we make a dictionary containing the per-basin metrics for each model.

In [5]:
#Retreive list of experiments with existing basin_metrics_files
metrics_list = [x for x in os.listdir(env_saves_dir) if x.startswith('basin')]

#Initiate dictionary to store all loaded cdfs files
metrics_dict = {}

#For every file in cdfs_list...
for file in metrics_list:
    
    #Define path to the file
    path = env_saves_dir / file
    
    #Get name from file name
    name = file.split('basin_metrics_')[1].split('.')[0]
    
    #Open path...
    with open(path,'rb') as f:
        
        #...and load
        metrics_dict[name] = pkl.load(f)

Here we make lists detailing what models are extreme and what models are random, as well as what types of experiments exist for each. Here we exclude the NWM benchmark runs -- they will be added later.

In [6]:
#Retrieve list of loaded models
models = list(metrics_dict.keys())
#If a model does not contain 'random' in the name, it is an extreme experiment
ext_models = [model for model in models if not 'random' in model and 'nwm_' not in model]
#If a model contains 'random' in the name, it is a random experiment
rand_models = [model for model in models if 'random' in model and 'nwm_' not in model]

In [7]:
#Retrieve the keys for an example extreme model, which constitutes the avaliable extreme experiments
ext_exps = list(metrics_dict[ext_models[0]].keys())
#Only use experiments for the defined climate index
ext_exps = [experiment for experiment in ext_exps if index in experiment]
#Retrieve the keys for an example random model, which constitutes the avaliable random experiments
rand_exps = list(metrics_dict[rand_models[0]].keys())
#Only use experiments for the defined climate index
metrics = list(metrics_dict[models[0]][ext_exps[0]].columns)

### Median Metrics Tables

##### Extreme Model Table

In [8]:
#Create multiindex for extreme models and experiments
exp_multidex = pd.MultiIndex.from_product([ext_models,ext_exps])

#Create dataframe with extreme multiindex
ext_median_df = pd.DataFrame(columns=metrics,index=exp_multidex)

#For every extreme model...
for model in ext_models:
    
    #Get a list of that model's experiments
    experiments = list(metrics_dict[model].keys())
    
    #And for each of those experiments...
    for experiment in experiments:
        
        #If the defined index is in the experiment...
        if index in experiment:
            
            #Get a list of metrics
            metrics = list(metrics_dict[model][experiment].columns)
            
            #And for each of those metrics...
            for metric in metrics:
                
                #Calculate the median value of that metric and save it into ext_median_df
                ext_median_df.loc[model,experiment][metric] = np.median(metrics_dict[model][experiment][metric])

#Show extreme model median metrics table
ext_median_df.head()

alpha_nse  beta_kge  beta_nse  \
nh_dynamic_extreme_daymet_nwm aridity_high  0.868716  0.987979 -0.004954   
                              aridity_low   0.870547  0.973161 -0.012992   
nh_static_extreme_daymet_nwm  aridity_high  0.878733  1.022305  0.015431   
                              aridity_low   0.856995  0.937718  -0.03082   
nh_static_extreme_daymet_all  aridity_high  0.856041  1.044905  0.026248   

                                                 kge       mse       nse  \
nh_dynamic_extreme_daymet_nwm aridity_high  0.727424  0.494415  0.736661   
                              aridity_low   0.758301  2.088403  0.748234   
nh_static_extreme_daymet_nwm  aridity_high  0.746649  0.443082  0.741288   
                              aridity_low    0.75456  2.082557  0.750663   
nh_static_extreme_daymet_all  aridity_high  0.680178  0.432945  0.710859   

                                                rmse  pearsonr  
nh_dynamic_extreme_daymet_nwm aridity_high  0.703147  0.876347  
                              aridity_low   1.445131  0.876207  
nh_static_extreme_daymet_nwm  aridity_high  0.665644  0.879823  
                              aridity_low   1.443107  0.880899  
nh_static_extreme_daymet_all  aridity_high  0.657986   0.87075

##### Random Model Table (Median)

Here we create a table detailing the each **median** metric score for the 531 CAMELS basins. 

In [9]:
#Create multiindex for random models and experiments
rand_multidex = pd.MultiIndex.from_product([rand_models,rand_exps],names=['models','experiments'])

#Create dataframe with random multiindex
rand_median_df = pd.DataFrame(columns=metrics,index=rand_multidex)

#For every random model...
for model in rand_models:
    
    #Get a list of experiments
    experiments = list(metrics_dict[model].keys())
    
    #And for each of those experiments...
    for experiment in experiments:
        
        #And for each metric...
        for metric in metrics:
            
            #Calculate the median value of that metric and save it into ext_median_df
            rand_median_df.loc[model,experiment][metric] = np.median(metrics_dict[model][experiment][metric])

#Peek at rand_median_df
rand_median_df.head()

alpha_nse  beta_kge  beta_nse  \
models                       experiments                                 
nh_dynamic_random_daymet_all 0            0.890283  0.987786 -0.004893   
                             1             0.88598  0.987089 -0.006193   
                             2            0.904534  0.995917  -0.00084   
                             3            0.895389  0.980272 -0.010066   
                             4            0.891942  0.990021 -0.004867   

                                               kge       mse       nse  \
models                       experiments                                 
nh_dynamic_random_daymet_all 0             0.78015  1.174311  0.772925   
                             1            0.776459  1.080283  0.779533   
                             2            0.784439  1.143959  0.770243   
                             3            0.787509  1.070211  0.776493   
                             4            0.776818  1.171692  0.768021   

                                              rmse  pearsonr  
models                       experiments                      
nh_dynamic_random_daymet_all 0            1.083656  0.894566  
                             1            1.039367  0.893854  
                             2             1.06956  0.890405  
                             3             1.03451  0.894492  
                             4            1.082447  0.896063

##### Random Model Table (Mean of the Medians)

Here we take the mean of the median metric scores by model so that we have one set of descriptive metrics for each random model.

In [10]:
mean_rand_df = pd.DataFrame(columns = rand_median_df.columns,index=rand_models)

for model in rand_models:
    rand_list = []
    mean_rand_df.loc[model] = rand_median_df.loc[model].mean()

mean_rand_df.head()

,alpha_nse,beta_kge,beta_nse,kge,mse,nse,rmse,pearsonr
nh_dynamic_random_daymet_all,0.893626,0.988217,-0.005372,0.781075,1.128091,0.773443,1.061908,0.893876
nh_static_random_nldas_extended_nwm,0.9171,0.994189,-0.002159,0.865654,0.713633,0.862731,0.844622,0.932401
nh_dynamic_random_nldas_extended_nwm,0.908706,0.993338,-0.002495,0.838795,0.841304,0.834425,0.916966,0.919115
sacsma_random_daymet_all,0.809232,1.14624,0.081949,0.579378,2.003844,0.59443,1.415194,0.812554
nh_static_random_daymet_all,0.891557,0.988062,-0.005727,0.79685,1.062771,0.78263,1.030707,0.897106


##### Total Table

Here we concatenate the extreme and (mean) random median metrics tables into one big table.

In [11]:
#Combine the extreme and random median dfs
median_df = pd.concat([ext_median_df,mean_rand_df], ignore_index=False)

#Peek at combined df
median_df.head()

,alpha_nse,beta_kge,beta_nse,kge,mse,nse,rmse,pearsonr
"(nh_dynamic_extreme_daymet_nwm, aridity_high)",0.868716,0.987979,-0.004954,0.727424,0.494415,0.736661,0.703147,0.876347
"(nh_dynamic_extreme_daymet_nwm, aridity_low)",0.870547,0.973161,-0.012992,0.758301,2.088403,0.748234,1.445131,0.876207
"(nh_static_extreme_daymet_nwm, aridity_high)",0.878733,1.022305,0.015431,0.746649,0.443082,0.741288,0.665644,0.879823
"(nh_static_extreme_daymet_nwm, aridity_low)",0.856995,0.937718,-0.03082,0.75456,2.082557,0.750663,1.443107,0.880899
"(nh_static_extreme_daymet_all, aridity_high)",0.856041,1.044905,0.026248,0.680178,0.432945,0.710859,0.657986,0.87075
